In [26]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

Loading the model

In [ ]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="aboonaji/llama2finetune-v2", 
                                                   quantization_config=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=getattr(torch, "float16"), 
                                                    bnb_4bit_quant_type="nf4"))

llama_model.config.use_cache=False
llama_model.config.pretraining_tp=1


Loading the Tokenizer

In [7]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="aboonaji/llama2finetune-v2", 
                                                trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

Setting the training arguments

In [8]:
training_arguments = TrainingArguments(output_dir="/.results", 
                                       per_device_train_batch_size=4, 
                                       max_steps=100)

Creating the supervised fine tuning trainer

In [25]:
import torch

# Define and initialize your model and tokenizer here

# Move model and tokenizer to GPU (if necessary)
# llama_model = llama_model.to('cuda:0')
# llama_tokenizer = llama_tokenizer.to('cuda:0')

# Define and initialize SFTTrainer without the device argument
llama_sft_trainer = SFTTrainer(
    model=llama_model,
    args=training_arguments,
    train_dataset=load_dataset(path="aboonaji/wiki_medical_terms_llam2_format", split="train"),
    tokenizer=llama_tokenizer,
    peft_config=LoraConfig(task_type="CAUSAL_LM", r=64, lora_alpha=16, lora_dropout=0.1),
    dataset_text_field="text"
)


OutOfMemoryError: CUDA out of memory. Tried to allocate 500.00 MiB. GPU 0 has a total capacity of 5.77 GiB of which 237.44 MiB is free. Including non-PyTorch memory, this process has 4.72 GiB memory in use. Of the allocated memory 4.02 GiB is allocated by PyTorch, and 99.37 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Training the model

In [ ]:
llama_sft_trainer.train()

Chatting with the model

In [ ]:
user_prompt = ""
text_generation_pipeline = pipeline(task="text-generation", model=llama_model, tokenizer=llama_tokenizer, max_length=300)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])